In [ ]:
pip install transformers datasets sentencepiece accelerate torch


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd


In [ ]:


dataset=pd.read_csv('skill_title_dataset.csv')


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving skill_title_dataset.csv to skill_title_dataset.csv
User uploaded file "skill_title_dataset.csv" with length 32157 bytes


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(dataset, test_size=0.1, random_state=42)


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = []
    for desc in examples["Description"]:
        # Instruction-style input to handle multilingual text
        prompt = f"Translate to English and convert skill description into a short English job title: {desc}"
        inputs.append(prompt)
    targets = examples["Title"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True,padding="max_length")
    labels = tokenizer(targets, max_length=32, truncation=True,padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
# def preprocess_function(examples):
#     inputs = [f"Generate a short job title for: {desc}" for desc in examples["description"]]
#     model_inputs = tokenizer(inputs, max_length=64, truncation=True, padding="max_length")
#     labels = tokenizer(text_target=examples["title"], max_length=16, truncation=True, padding="max_length")
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:

training_args = TrainingArguments(
    output_dir="./flan-skilltitle-model",
    #evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=1,
    logging_steps=100,
    push_to_hub=False,
    report_to="none"
)


In [ ]:
# Step 7: Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer
)

trainer.train()


/tmp/ipython-input-258088343.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
100,1.792900
200,0.000800


Step,Training Loss
100,1.792900
200,0.000800


TrainOutput(global_step=285, training_loss=0.6294578071160798, metrics={'train_runtime': 3451.5933, 'train_samples_per_second': 0.652, 'train_steps_per_second': 0.083, 'total_flos': 385176010752000.0, 'train_loss': 0.6294578071160798, 'epoch': 5.0})

In [ ]:
!ls /content/flan-skilltitle-model


checkpoint-285		runs			 tokenizer.json
config.json		special_tokens_map.json  training_args.bin
generation_config.json	spiece.model
model.safetensors	tokenizer_config.json


In [ ]:
trainer.save_model("./flan-skilltitle-model")
tokenizer.save_pretrained("./flan-skilltitle-model")


('./flan-skilltitle-model/tokenizer_config.json',
 './flan-skilltitle-model/special_tokens_map.json',
 './flan-skilltitle-model/spiece.model',
 './flan-skilltitle-model/added_tokens.json',
 './flan-skilltitle-model/tokenizer.json')

In [ ]:
!ls -lh


total 3.3G
drwxr-xr-x 4 root root 4.0K Oct 31 17:09 flan-skilltitle-model
-rw-r--r-- 1 root root 3.3G Oct 31 17:12 flan-skilltitle-model.zip
drwxr-xr-x 1 root root 4.0K Oct 29 13:38 sample_data
-rw-r--r-- 1 root root  32K Oct 31 15:59 skill_title_dataset.csv


In [ ]:
from google.colab import files
!zip -r flan-skilltitle-model.zip flan-skilltitle-model
files.download("flan-skilltitle-model.zip")


updating: flan-skilltitle-model/ (stored 0%)
updating: flan-skilltitle-model/generation_config.json (deflated 27%)
updating: flan-skilltitle-model/checkpoint-285/ (stored 0%)
updating: flan-skilltitle-model/checkpoint-285/generation_config.json (deflated 27%)
updating: flan-skilltitle-model/checkpoint-285/special_tokens_map.json (deflated 85%)
updating: flan-skilltitle-model/checkpoint-285/tokenizer_config.json (deflated 95%)
updating: flan-skilltitle-model/checkpoint-285/scheduler.pt (deflated 62%)
updating: flan-skilltitle-model/checkpoint-285/optimizer.pt (deflated 17%)
updating: flan-skilltitle-model/checkpoint-285/model.safetensors (deflated 7%)
updating: flan-skilltitle-model/checkpoint-285/spiece.model (deflated 48%)
updating: flan-skilltitle-model/checkpoint-285/rng_state.pth (deflated 26%)
updating: flan-skilltitle-model/checkpoint-285/trainer_state.json (deflated 57%)
updating: flan-skilltitle-model/checkpoint-285/training_args.bin (deflated 53%)
updating: flan-skilltitle-mod

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("./flan-skilltitle-model")
model = AutoModelForSeq2SeqLM.from_pretrained("./flan-skilltitle-model")


In [ ]:
# def generate_title(description):
#     input_text = f"Convert this person's work description into a short English job title Description: {description}"
#     inputs = tokenizer(input_text, return_tensors="pt")
#     outputs = model.generate(**inputs, max_length=32, num_beams=5)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# print(generate_title("मैं खाना बनाता हूं और ग्राहकों को परोसता हूं"))


In [ ]:
# print(generate_title("मैं खाना बनाता हूं और ग्राहकों को परोसता हूं"))  # Expected: Cook or Waiter
# print(generate_title("Yo reparo muebles rotos y hago sillas de madera"))  # Expected: Carpenter
# print(generate_title("I design embroidery on sarees"))  # Expected: Embroidery Designer


In [ ]:
from huggingface_hub import login
login()


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="./flan-skilltitle-model",  # Your trained model folder
    repo_id="Divya21026/flan-skilltitle-model",  # Choose repo name
    repo_type="model"
)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...eckpoint-285/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

  ...eckpoint-285/optimizer.pt:   0%|          | 9.22MB / 1.98GB            

  ...eckpoint-285/spiece.model: 100%|##########|  792kB /  792kB            

...

  ...ckpoint-285/rng_state.pth:   9%|8         | 1.29kB / 14.5kB            

  ...int-285/training_args.bin:   9%|8         |   520B / 5.84kB            

  ...26983.ec48325c0f88.1218.0:   9%|8         |   527B / 5.92kB            

  ...e-model/training_args.bin:   9%|8         |   520B / 5.84kB            

CommitInfo(commit_url='https://huggingface.co/Divya21026/flan-skilltitle-model/commit/45b6a59944d94280b6260512c72d0943c514c028', commit_message='Upload folder using huggingface_hub', commit_description='', oid='45b6a59944d94280b6260512c72d0943c514c028', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Divya21026/flan-skilltitle-model', endpoint='https://huggingface.co', repo_type='model', repo_id='Divya21026/flan-skilltitle-model'), pr_revision=None, pr_num=None)